In [4]:
# Load libraries
# library(DiffBind)
library(BiocParallel)
library(dplyr)
library(TxDb.Athaliana.BioMart.plantsmart51)
# library(ChIPseeker)

library(chromVAR)
library(motifmatchr)
library(BiocParallel)
register(MulticoreParam(50)) # Use 50 cores
# Set options
options(stringsAsFactors = F)
Fold_cutoff <- 1
FDR_cutoff <- 0.05



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: GenomicFeatures

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Lo

In [5]:
map_id_name <- read.csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')
map_id_name <- map_id_name %>% dplyr::rename('gene_name' = 'external_gene_name', 'gene_id' = 'ensembl_gene_id')
sample_n(map_id_name, 2)

gene_id,gene_name
<chr>,<chr>
AT4G32070,Phox4
AT5G62700,TUB3


In [36]:
map_id_name_mutated <- map_id_name %>% dplyr::select(2, 1) %>% mutate(gene_name = paste(gene_name, gene_id, sep = '_'))

In [37]:
# Assuming map_id_name is your data frame
write.table(map_id_name_mutated,
            file = './TOBIAS/meme/motif2gene_mapping.txt', 
            sep = "\t", # Separator (tab in this case)
            row.names = FALSE, # Exclude row names
            col.names = FALSE, # Exclude column headers
            quote = FALSE) # Exclude quotes


In [7]:
sample_n(map_id_name, 3)

gene_id,gene_name
<chr>,<chr>
AT1G49005,CLE11
AT5G65165,SDH2-3
AT4G18900,AT4G18900


In [10]:
# Assuming map_id_name is your data.frame
gene_name_map <- setNames(map_id_name$gene_name, map_id_name$gene_id)

In [17]:
gene_name_map[1:20]

AT5G16970   AT4G32100   AT2G43120   AT1G30814   AT3G18710   AT4G25880 
      "AER" "AT4G32100" "AT2G43120" "AT1G30814"     "PUB29"      "PUM6" 
  AT3G07950   AT4G03450   AT1G71695   AT1G12980   AT5G41480   AT5G15008 
"AT3G07950" "AT4G03450" "AT1G71695"      "ESR1"      "GLA1" "AT5G15008" 
  AT3G51870   AT4G29950   AT3G18310   AT5G07360   AT4G08535   AT4G26490 
"AT3G51870" "AT4G29950" "AT3G18310" "AT5G07360" "AT4G08535" "AT4G26490" 
  AT5G46240   AT4G16955 
     "KAT1" "AT4G16955"

In [12]:
for (i in names(Ath_motifs_list)) {
    motif_id <- slot(Ath_motifs_list[[i]], "ID")
    if (motif_id %in% names(gene_name_map)) {
        slot(Ath_motifs_list[[i]], "name") <- gene_name_map[[motif_id]]
    }
}


In [22]:
# Assuming gene_name_map is a named vector
# Ensure the mapping is correct
print(head(gene_name_map))

# Update the IDs in Ath_motifs_list
for (i in seq_along(Ath_motifs_list)) {
    motif_id <- slot(Ath_motifs_list[[i]], "ID")
    if (motif_id %in% names(gene_name_map)) {
        # Update the ID slot
        new_motif <- Ath_motifs_list[[i]]
        slot(new_motif, "ID") <- gene_name_map[[motif_id]]
        # Assign the updated motif back to the list
        Ath_motifs_list[[i]] <- new_motif
    }
}

# Check a few entries to verify the changes
Ath_motifs_list %>% str


  AT5G16970   AT4G32100   AT2G43120   AT1G30814   AT3G18710   AT4G25880 
      "AER" "AT4G32100" "AT2G43120" "AT1G30814"     "PUB29"      "PUM6" 
Formal class 'PFMatrixList' [package "TFBSTools"] with 4 slots
  ..@ listData       :List of 619
  .. ..$ AT1G01060:Formal class 'PFMatrix' [package "TFBSTools"] with 7 slots
  .. .. .. ..@ ID           : chr "MP00119"
  .. .. .. ..@ name         : chr "AT1G01060"
  .. .. .. ..@ matrixClass  : chr "Unknown"
  .. .. .. ..@ strand       : chr "*"
  .. .. .. ..@ bg           : Named num [1:4] 0.332 0.168 0.168 0.332
  .. .. .. .. ..- attr(*, "names")= chr [1:4] "A" "C" "G" "T"
  .. .. .. ..@ tags         :List of 1
  .. .. .. .. ..$ eval.string: chr "4.8e-803"
  .. .. .. ..@ profileMatrix: int [1:4, 1:10] 512 1 31 55 0 0 599 0 599 0 ...
  .. .. .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. .. .. ..$ : chr [1:4] "A" "C" "G" "T"
  .. .. .. .. .. ..$ : chr [1:10] "A" "G" "A" "T" ...
  .. ..$ AT1G01250:Formal class 'PFMatrix' [package "TFBSTool

In [24]:
Ath_motifs_list[1]

PFMatrixList of length 1
names(1): AT1G01060

In [27]:
gene_name_map['AT1G01060']

AT1G01060 
    "LHY"

In [28]:
# Path to the original and new MEME files
meme_file <- "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"
new_meme_file <- "./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme"

# Read the MEME file and process it
lines <- readLines(meme_file)
new_lines <- vector("list", length(lines))
for (i in 1:length(lines)) {
    line <- lines[i]
    if (startsWith(line, "MOTIF")) {
        # Extract the motif information
        motif_info <- unlist(strsplit(line, " "))
        gene_id <- motif_info[2]
        
        # Check if the gene_id is in the gene_name_map and update
        if (gene_id %in% names(gene_name_map)) {
            new_motif_id <- gene_name_map[[gene_id]]
            line <- paste("MOTIF", gene_id, new_motif_id)
        }
    }
    new_lines[[i]] <- line
}

# Write the updated content to the new MEME file
writeLines(unlist(new_lines), new_meme_file)
